<a href="https://colab.research.google.com/github/ManvithaReddy1133/ecommerce-data-science/blob/main/Manvitha_Indukuri_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
merged_data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

In [5]:
import pandas as pd
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

In [11]:
customer_features = merged_data.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'Category': 'nunique'
}).reset_index()

# Rename columns for clarity
customer_features.rename(columns={'Category': 'UniqueCategories'}, inplace=True)


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_scaled = scaler.fit_transform(customer_features.iloc[:, 1:])


In [13]:
from sklearn.neighbors import NearestNeighbors

# Initialize and train the model
model = NearestNeighbors(n_neighbors=4, metric='cosine')
model.fit(features_scaled)

# Find nearest neighbors
distances, indices = model.kneighbors(features_scaled)


In [14]:
# Prepare recommendations
recommendations = {}

for i, customer_id in enumerate(customer_features['CustomerID']):
    similar_customers = [
        (customer_features['CustomerID'][indices[i][j]], 1 - distances[i][j])
        for j in range(1, 4)  # Skip self (first neighbor)
    ]
    recommendations[customer_id] = similar_customers


In [15]:
# Save recommendations to CSV
import csv

output_file = 'Lookalike.csv'

with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['CustomerID', 'SimilarCustomers'])

    for customer, similar_list in recommendations.items():
        similar_str = "; ".join([f"{sim[0]}:{sim[1]:.4f}" for sim in similar_list])
        writer.writerow([customer, similar_str])

print(f'Recommendations saved to {output_file}')


Recommendations saved to Lookalike.csv
